In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, mixed_precision
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, AveragePooling2D,
                                     GlobalAveragePooling2D, BatchNormalization,
                                     Activation, Add, Concatenate, Dense, Dropout,
                                     Multiply, Reshape)
import os, kagglehub

# Disable XLA JIT (to avoid huge pinned‐host allocs)
tf.config.optimizer.set_jit(False)

# GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for g in gpus:
        tf.config.experimental.set_memory_growth(g, True)
    print(" GPU memory growth enabled")
else:
    print(" No GPU detected")

mixed_precision.set_global_policy('mixed_float16')
print(" Mixed precision policy:", mixed_precision.global_policy())


 GPU memory growth enabled
 Mixed precision policy: <DTypePolicy "mixed_float16">


In [ ]:
import os
import kagglehub

path = kagglehub.dataset_download("nafishamoin/bangladeshi-crops-disease-dataset")
print(" Raw dataset path:", path)

main_folder = os.listdir(path)[0]
main_path = os.path.join(path, main_folder)
print(" Main folder:", main_folder)
print("Contents of main folder:", os.listdir(main_path)[:5])

image_root = os.path.join(main_path, "Crop___DIsease")
print(" Image root folder:", image_root)

classes = os.listdir(image_root)
print(" Classes found:", classes[:10])

 Raw dataset path: /kaggle/input/bangladeshi-crops-disease-dataset
 Main folder: CropDisease
Contents of main folder: ['Crop___DIsease']
 Image root folder: /kaggle/input/bangladeshi-crops-disease-dataset/CropDisease/Crop___DIsease
 Classes found: ['Wheat___Yellow_Rust', 'Potato___Healthy', 'Corn___Healthy', 'Wheat___Brown_Rust', 'Corn___Gray_Leaf_Spot', 'Rice___Brown_Spot', 'Wheat___Healthy', 'Rice___Leaf_Blast', 'Potato___Late_Blight', 'Rice___Healthy']


In [ ]:
data_root = os.path.join(main_path, "Crop___DIsease")

In [ ]:
#Load datasets with one hot labels
import tensorflow as tf

batch_size = 8
img_size   = (160, 160)

dataset_train = tf.keras.preprocessing.image_dataset_from_directory(
    data_root,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)
dataset_val = tf.keras.preprocessing.image_dataset_from_directory(
    data_root,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

print("Train element spec:", dataset_train.element_spec)

Found 31053 files belonging to 15 classes.
Using 24843 files for training.
Found 31053 files belonging to 15 classes.
Using 6210 files for validation.
Train element spec: (TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 15), dtype=tf.float32, name=None))


In [ ]:
num_classes = dataset_train.element_spec[1].shape[-1]
print("Detected num_classes =", num_classes)

Detected num_classes = 15


In [ ]:
#Preprocessing
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = (
    dataset_train
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .shuffle(500)
    .prefetch(1)
)

val_ds = (
    dataset_val
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .prefetch(1)
)

In [ ]:
import tensorflow as tf
import math
from tensorflow.keras import layers, regularizers, backend as K, Model
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, Activation,
    MaxPooling2D, AveragePooling2D,
    GlobalAveragePooling2D, Dense, Dropout, Concatenate,Reshape, Dot, Add, Lambda
)

In [ ]:
def rsa_attention(x, reg_l2=1e-4, reg_l1=1e-4, name=None):
    C = x.shape[-1]
    H = x.shape[1]
    W = x.shape[2]
    N = H * W

    f = Conv2D(C,1,padding='same',use_bias=True,
               kernel_regularizer=regularizers.l2(reg_l2),
               bias_regularizer   =regularizers.l1(reg_l1),
               name=(name+"_f" if name else None))(x)
    g = Conv2D(C,1,padding='same',use_bias=True,
               kernel_regularizer=regularizers.l2(reg_l2),
               bias_regularizer   =regularizers.l1(reg_l1),
               name=(name+"_g" if name else None))(x)
    h = Conv2D(C,1,padding='same',use_bias=True,
               kernel_regularizer=regularizers.l2(reg_l2),
               bias_regularizer   =regularizers.l1(reg_l1),
               name=(name+"_h" if name else None))(x)

    f_flat = Reshape((N, C), name=(name+"_reshape_f" if name else None))(f)
    g_flat = Reshape((N, C), name=(name+"_reshape_g" if name else None))(g)
    h_flat = Reshape((N, C), name=(name+"_reshape_h" if name else None))(h)

    scores = Dot(axes=(2,2), name=(name+"_score" if name else None))([f_flat, g_flat])

    inv_sqrt_C = 1.0 / math.sqrt(C)
    scores = Lambda(lambda t: t * inv_sqrt_C,
                    name=(name+"_scale" if name else "rsa_scale"))(scores)

    alpha = Activation('softmax', name=(name+"_softmax" if name else "rsa_softmax"))(scores)
    o_flat = Dot(axes=(2,1), name=(name+"_apply" if name else "rsa_apply"))([alpha, h_flat])
    o = Reshape((H, W, C), name=(name+"_out" if name else "rsa_out"))(o_flat)
    return o

In [ ]:
#Model building blocks

def residual_block(x, filters, stride=1):
    shortcut = x
    x = Conv2D(filters,3,strides=stride,padding='same',use_bias=False)(x)
    x = BatchNormalization()(x); x=Activation('relu')(x)
    x = Conv2D(filters,3,padding='same',use_bias=False)(x)
    x = BatchNormalization()(x)
    if stride!=1 or shortcut.shape[-1]!=filters:
        shortcut = Conv2D(filters,1,strides=stride,use_bias=False)(shortcut)
        shortcut = BatchNormalization()(shortcut)
    x = Add()([x,shortcut]); return Activation('relu')(x)

def residual_block_group(x, f, n, stride=1):
    x = residual_block(x,f,stride)
    for _ in range(1,n):
        x = residual_block(x,f,1)
    return x

def dense_block(x, layers_n, g_rate):
    feats = [x]
    for _ in range(layers_n):
        y = Concatenate()(feats)
        y = BatchNormalization()(y); y=Activation('relu')(y)
        y = Conv2D(4*g_rate,1,padding='same',use_bias=False)(y)
        y = BatchNormalization()(y); y=Activation('relu')(y)
        y = Conv2D(g_rate,3,padding='same',use_bias=False)(y)
        feats.append(y)
    return Concatenate()(feats)

def transition_layer(x):
    x = BatchNormalization()(x)
    x = Conv2D(x.shape[-1]//2,1,padding='same',use_bias=False)(x)
    return AveragePooling2D(2,2)(x)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras import callbacks, optimizers, losses
import os
from google.colab import drive



def build_rsa_derefnet(input_shape=(160,160,3), num_classes=15, growth_rate=12):
    i = Input(shape=input_shape)

    # Initial conv
    x0 = Conv2D(32,3,padding='same',use_bias=False)(i)
    x0 = BatchNormalization()(x0); x0 = Activation('relu')(x0)

    # Residual stream with RSA
    r1 = residual_block_group(x0,  64, 1, stride=1)
    p1 = AveragePooling2D(pool_size=4)(r1)
    a1 = rsa_attention(p1, name="res1")
    r2 = residual_block_group(a1, 128, 1, stride=2)
    p2 = AveragePooling2D(pool_size=2)(r2)
    a2 = rsa_attention(p2, name="res2")
    r3 = residual_block_group(a2, 256, 1, stride=2)
    g_r = GlobalAveragePooling2D()(r3)

    # Dense stream with RSA
    d1 = Conv2D(32,3,padding='same',use_bias=False)(i)
    d1 = BatchNormalization()(d1); d1 = Activation('relu')(d1)
    d1 = MaxPooling2D(2)(d1)
    db1 = dense_block(d1, 3, growth_rate)
    t1 = transition_layer(db1)
    a3 = rsa_attention(t1, name="dens1")
    db2 = dense_block(a3, 3, growth_rate)
    t2 = transition_layer(db2)
    a4 = rsa_attention(t2, name="dens2")
    db3 = dense_block(a4, 5, growth_rate)
    g_d = GlobalAveragePooling2D()(db3)

    # Fuse & classify
    f = Concatenate()([g_r, g_d])
    f = Dropout(0.5)(f)
    out = Dense(num_classes, activation='softmax', dtype='float32')(f)

    return Model(i, out)

# Learning rate schedule
lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True
)

# Build and compile the model
model = build_rsa_derefnet(
    input_shape=(160,160,3),
    num_classes=num_classes,
    growth_rate=12
)
from tensorflow.keras import optimizers, losses

steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
total_steps = steps_per_epoch * 30

cosine = optimizers.schedules.CosineDecay(
    initial_learning_rate=3e-4, decay_steps=total_steps
)

opt = optimizers.AdamW(learning_rate=cosine, weight_decay=1e-4, clipnorm=1.0)

model.compile(
    optimizer=opt,
    loss=losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)





history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
)


Epoch 1/30
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 374s 85ms/step - accuracy: 0.6626 - loss: 1.5954 - val_accuracy: 0.8095 - val_loss: 1.1222
Epoch 2/30
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 253s 62ms/step - accuracy: 0.8775 - loss: 0.9790 - val_accuracy: 0.8662 - val_loss: 0.9938
Epoch 3/30
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 214s 62ms/step - accuracy: 0.9072 - loss: 0.8942 - val_accuracy: 0.9045 - val_loss: 0.8378
Epoch 4/30
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 217s 63ms/step - accuracy: 0.9193 - loss: 0.8550 - val_accuracy: 0.9233 - val_loss: 0.8166
Epoch 5/30
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 216s 62ms/step - accuracy: 0.9294 - loss: 0.8227 - val_accuracy: 0.9242 - val_loss: 0.7913
Epoch 6/30
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 264s 63ms/step - accuracy: 0.9363 - loss: 0.7956 - val_accuracy: 0.9188 - val_loss: 0.7957
Epoch 7/30
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 261s 62ms/step - accuracy: 0.9416 - loss: 0.7740 - val_accuracy: 0.9477 - val_loss: 0.7415
Epoch 8/30
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 218s 63ms/step - accuracy: 

In [ ]:
import matplotlib.pyplot as plt

# Function to plot training and validation metrics
def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(14, 5))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo-', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'go-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'bo-', label='Training Loss')
    plt.plot(epochs, val_loss, 'go-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

plot_training_history(history)
